In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

In [2]:
covid_data_path = "resources/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time_-_ARCHIVED.csv"

# Read the mouse data and the study results
covid_data_df = pd.read_csv(covid_data_path)

covid_data_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0,03/12/2021 03:20:13 PM,Agree,NaN
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0,12/02/2021 02:35:20 PM,Agree,Not agree
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0,01/03/2022 03:18:16 PM,NaN,NaN
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0,11/22/2021 12:00:00 AM,Agree,Agree
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0,05/31/2022 01:20:20 PM,NaN,NaN


In [3]:
clean_df = covid_data_df.drop(columns=["created_at", "consent_cases", "consent_deaths"])
clean_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0


In [4]:
clean_df = clean_df.rename(columns={"submission_date":"Submission Date", "state":"State", "tot_cases":"Total Cases",
                        "conf_cases":"Confirmed Cases", "prob_cases":"Probable Cases", "new_case":"New Cases",
                        "pnew_case":"Probable New Cases", "tot_death":"Total Deaths", "conf_death":"Confirmed Deaths",
                        "prob_death": "Probable Deaths", "new_death":"New Deaths", "pnew_death":"Probable New Deaths"})
clean_df.head()

,Submission Date,State,Total Cases,Confirmed Cases,Probable Cases,New Cases,Probable New Cases,Total Deaths,Confirmed Deaths,Probable Deaths,New Deaths,Probable New Deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,NaN,NaN,0,0.0
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,NaN,NaN,9,0.0
2,01/02/2022,AS,11,NaN,NaN,0,0.0,0,NaN,NaN,0,0.0
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0
4,05/30/2022,AK,251425,NaN,NaN,0,0.0,1252,NaN,NaN,0,0.0


In [5]:
clean_df = clean_df.fillna(0)
clean_df

,Submission Date,State,Total Cases,Confirmed Cases,Probable Cases,New Cases,Probable New Cases,Total Deaths,Confirmed Deaths,Probable Deaths,New Deaths,Probable New Deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,0.0,0.0,0,0.0
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,0.0,0.0,9,0.0
2,01/02/2022,AS,11,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0
4,05/30/2022,AK,251425,0.0,0.0,0,0.0,1252,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60055,02/09/2021,TX,2504556,0.0,0.0,13329,2676.0,43306,0.0,0.0,207,0.0
60056,11/20/2020,FL,913561,0.0,0.0,8217,1677.0,19014,0.0,0.0,79,5.0
60057,08/17/2020,NM,23500,0.0,0.0,92,0.0,682,0.0,0.0,4,0.0
60058,06/17/2020,MS,24223,24038.0,185.0,521,6.0,1191,1172.0,19.0,9,0.0


In [6]:
clean_df['State'] = clean_df['State'].replace({'NYC': 'NY'})
clean_df

,Submission Date,State,Total Cases,Confirmed Cases,Probable Cases,New Cases,Probable New Cases,Total Deaths,Confirmed Deaths,Probable Deaths,New Deaths,Probable New Deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,0.0,0.0,0,0.0
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,0.0,0.0,9,0.0
2,01/02/2022,AS,11,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0
4,05/30/2022,AK,251425,0.0,0.0,0,0.0,1252,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60055,02/09/2021,TX,2504556,0.0,0.0,13329,2676.0,43306,0.0,0.0,207,0.0
60056,11/20/2020,FL,913561,0.0,0.0,8217,1677.0,19014,0.0,0.0,79,5.0
60057,08/17/2020,NM,23500,0.0,0.0,92,0.0,682,0.0,0.0,4,0.0
60058,06/17/2020,MS,24223,24038.0,185.0,521,6.0,1191,1172.0,19.0,9,0.0


In [13]:
# consider the list
list1 = ["AS","RMI","PR","PW","GU","MP","FSM","VI"]


filtered_df = clean_df[(clean_df['State'] != "AS") & (clean_df['State'] != "RMI") &
                      (clean_df['State'] != "PR") & (clean_df['State'] != "PW") & 
                      (clean_df['State'] != "GU") & (clean_df['State'] != "MP") &
                      (clean_df['State'] != "FSM") & (clean_df['State'] != "VI")]
filtered_df



,Submission Date,State,Total Cases,Confirmed Cases,Probable Cases,New Cases,Probable New Cases,Total Deaths,Confirmed Deaths,Probable Deaths,New Deaths,Probable New Deaths
0,03/11/2021,KS,297229,241035.0,56194.0,0,0.0,4851,0.0,0.0,0,0.0
1,12/01/2021,ND,163565,135705.0,27860.0,589,220.0,1907,0.0,0.0,9,0.0
3,11/22/2021,AL,841461,620483.0,220978.0,703,357.0,16377,12727.0,3650.0,7,3.0
4,05/30/2022,AK,251425,0.0,0.0,0,0.0,1252,0.0,0.0,0,0.0
6,04/03/2020,ND,173,0.0,0.0,14,0.0,3,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60055,02/09/2021,TX,2504556,0.0,0.0,13329,2676.0,43306,0.0,0.0,207,0.0
60056,11/20/2020,FL,913561,0.0,0.0,8217,1677.0,19014,0.0,0.0,79,5.0
60057,08/17/2020,NM,23500,0.0,0.0,92,0.0,682,0.0,0.0,4,0.0
60058,06/17/2020,MS,24223,24038.0,185.0,521,6.0,1191,1172.0,19.0,9,0.0
